## Imports

In [3]:
import requests 
import apikey
import json
import pandas as pd
import san

## Getting Data

Getting the first 50 cryptocurrencies from the **coinmarketcap API**.

In [2]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'50',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': apikey.key
}


In [3]:
json = requests.get(url,params = parameters, headers = headers).json()

In [4]:
coins = json['data']

In [5]:
crypto_data = pd.DataFrame(columns=['id', 'name', 'ticker', 'price', 'supply', 'platform'])

In [6]:
for x in coins:
    
    id_num = x['id']
    name = x['name']
    symbol = x['symbol']
    price = x['quote']['USD']['price']
    supply = x['total_supply']
    platform = x['platform']
    
    crypto_data = crypto_data.append({'id':id_num, 'name':name, 'ticker':symbol, 'price':price, 'supply':supply,
                                      'platform':platform}, ignore_index=True)

In [7]:
# if the platform is null, I can sort out layer 2 solutions

crypto_data = crypto_data[crypto_data["platform"].isnull()].reset_index(drop=True)

In [8]:
crypto_data

,id,name,ticker,price,supply,platform
0,1,Bitcoin,BTC,63090.909717,18861918,None
1,1027,Ethereum,ETH,4502.708780,1.18182e+08,None
2,1839,Binance Coin,BNB,559.366786,166801148,None
3,2010,Cardano,ADA,1.968280,3.36769e+10,None
4,5426,Solana,SOL,215.788783,5.07977e+08,None
5,52,XRP,XRP,1.125273,99990181120,None
6,6636,Polkadot,DOT,51.233964,1.1033e+09,None
7,74,Dogecoin,DOGE,0.275067,1.31957e+11,None
8,4172,Terra,LUNA,44.570800,9.69108e+08,None
9,5805,Avalanche,AVAX,67.172343,3.91128e+08,None


Gathering the social media and development data from the **sanbase API**.

In [9]:
projects = san.get("projects/all")

In [10]:
projects.tail()

,marketSegment,name,slug,ticker,totalSupply
2022,DeFi,PieDAO DOUGH v2,piedao-dough-v2,DOUGH,85347346.3651403
2023,DeFi,SmartKey,smartkey,SKEY,1000000000
2024,Privacy,Firo,zcoin,XZC,12427854.74033085
2025,Financial,Crypto.com Coin,crypto-com-coin,CRO,30263013692
2026,Insurance,Lossless,lossless,LSS,100000000


In [11]:
# Merging the 2 dataframes to get the slugs for the sanbase API

merged = crypto_data.merge(projects, on=['ticker'], how='left')

In [12]:
index_selection = pd.DataFrame(columns=['id', 'ticker', 'name', 'slug', 'market_cap_90d',
                                        'volume_90d', 'social_media_90d', 'github_90d'])

In [13]:
# index_selection df shows the 90 day average of the key metrics to rank the constituents

for ticker in merged['slug']:
    
    mc = san.get('marketcap_usd/' + str(ticker))[-90:].mean()[0]
    vol = san.get("volume_usd/" + str(ticker))[-90:].mean()[0]
    soc = san.get("social_volume_total/" + str(ticker))[-90:].mean()[0]
    gh = san.get('github_activity/' + str(ticker))[-90:].mean()[0]
    
    
    index_selection = index_selection.append({'slug':ticker,'market_cap_90d':mc, 'volume_90d':vol, 'social_media_90d':soc,
                                              'github_90d':gh }, ignore_index=True)
    


In [14]:
index_selection['id']  = merged['id']
index_selection['name'] = merged['name_x']
index_selection['ticker'] = merged['ticker']

In [15]:
index_selection.head()

,id,ticker,name,slug,market_cap_90d,volume_90d,social_media_90d,github_90d
0,1,BTC,Bitcoin,bitcoin,9.484253e+11,3.586698e+10,14088.211111,156.966667
1,1027,ETH,Ethereum,ethereum,4.066554e+11,2.091694e+10,7385.444444,444.877778
2,1839,BNB,Binance Coin,binance-coin,7.266373e+10,2.099380e+09,1860.622222,4.888889
3,2010,ADA,Cardano,cardano,7.289835e+10,4.360350e+09,3324.766667,389.188889
4,5426,SOL,Solana,solana,3.903487e+10,3.457247e+09,2347.800000,319.222222


## Data Transformation

#### Normalization

In [16]:

index_selection_norm = index_selection.copy()

In [17]:
def normalize_column(dataframe, column):
    
    dataframe[column] = (dataframe[column] - dataframe[column].min()) / (dataframe[column].max() - dataframe[column].min())

In [18]:
def normalize_dataframe(dataframe):
    
    normalize_column(dataframe, 'market_cap_90d')
    normalize_column(dataframe, 'volume_90d')
    normalize_column(dataframe, 'social_media_90d')
    normalize_column(dataframe, 'github_90d')
    
    return dataframe

In [19]:
normalize_dataframe(index_selection_norm)

,id,ticker,name,slug,market_cap_90d,volume_90d,social_media_90d,github_90d
0,1,BTC,Bitcoin,bitcoin,1.000000,1.000000,1.000000,0.352750
1,1027,ETH,Ethereum,ethereum,0.427682,0.581782,0.524007,1.000000
2,1839,BNB,Binance Coin,binance-coin,0.074858,0.055374,0.131665,0.010866
3,2010,ADA,Cardano,cardano,0.075106,0.118623,0.235641,0.874806
4,5426,SOL,Solana,solana,0.039333,0.093359,0.166262,0.717515
5,52,XRP,XRP,ripple,0.051244,0.125290,0.067044,0.128791
6,6636,DOT,Polkadot,polkadot-new,0.031388,0.058847,0.061784,0.897987
7,74,DOGE,Dogecoin,dogecoin,0.033637,0.077145,0.216133,0.082055
8,4172,LUNA,Terra,luna,0.012341,0.030446,0.032519,0.095419
9,5805,AVAX,Avalanche,avalanche,0.009303,0.025671,0.026794,0.089049


#### Selection Score

In [20]:
index_score = index_selection_norm.copy()

In [21]:
# Creating a score form the normalized features

index_score['selection_score'] = index_score['market_cap_90d'] + index_score['volume_90d'] + index_score['social_media_90d'] + index_score['github_90d']

In [22]:
# Ranking by the score and selecting the top 10

final_selection = index_score.sort_values(by='selection_score', ascending=False).head(10).reset_index(drop=True)

In [23]:
final_selection

,id,ticker,name,slug,market_cap_90d,volume_90d,social_media_90d,github_90d,selection_score
0,1,BTC,Bitcoin,bitcoin,1.000000,1.000000,1.000000,0.352750,3.352750
1,1027,ETH,Ethereum,ethereum,0.427682,0.581782,0.524007,1.000000,2.533472
2,2010,ADA,Cardano,cardano,0.075106,0.118623,0.235641,0.874806,1.304176
3,6636,DOT,Polkadot,polkadot-new,0.031388,0.058847,0.061784,0.897987,1.050006
4,5426,SOL,Solana,solana,0.039333,0.093359,0.166262,0.717515,1.016470
5,2280,FIL,Filecoin,file-coin,0.006138,0.027689,0.004689,0.494480,0.532996
6,6535,NEAR,NEAR Protocol,near-protocol,0.001645,0.005196,0.036520,0.404082,0.447441
7,4558,FLOW,Flow,flow,0.000000,0.002701,0.008718,0.418444,0.429863
8,74,DOGE,Dogecoin,dogecoin,0.033637,0.077145,0.216133,0.082055,0.408971
9,3794,ATOM,Cosmos,cosmos,0.004890,0.025588,0.032284,0.341859,0.404621


#### Base Data & Index level

In [65]:
score_sum = final_selection['selection_score'].sum()

In [70]:
# A df to use every time I want to calculate index level

base_data = pd.DataFrame(columns=['id', 'ticker', 'name', 'weight'])

In [71]:
base_data['weight'] = final_selection['selection_score'] / score_sum

In [72]:
base_data['id']  = final_selection['id']
base_data['name'] = final_selection['name']
base_data['ticker'] = final_selection['ticker']

In [75]:
base_metrics = crypto_data[['id', 'price', 'supply']]

In [78]:
base_data = base_data.merge(base_metrics, on='id', how='left')

In [10]:
base_data

,id,ticker,name,weight,price,supply
0,1,BTC,Bitcoin,0.291830,60320.597160,1.885981e+07
1,1027,ETH,Ethereum,0.220825,4194.338042,1.181508e+08
2,2010,ADA,Cardano,0.114100,1.931899,3.365611e+10
3,6636,DOT,Polkadot,0.091312,41.474665,1.103303e+09
4,5426,SOL,Solana,0.087473,187.765825,5.077740e+08
5,2280,FIL,Filecoin,0.046785,60.364519,1.187322e+08
6,6535,NEAR,NEAR Protocol,0.039328,10.035997,1.000000e+09
7,4558,FLOW,Flow,0.037572,13.846497,1.378212e+09
8,3794,ATOM,Cosmos,0.035395,35.822447,2.815780e+08
9,74,DOGE,Dogecoin,0.035380,0.259176,1.319160e+11


In [ ]:
base_data.to_csv(r'base_data.csv)

In [32]:
# df to store the index values

index_levels = pd.DataFrame(columns=['Date', 'Index'])

In [33]:
# starting value

first_row = {'Date':'2021-10-31', "Index":100}

In [ ]:
# adding satrting value to df

index_levels = index_levels.append(first_row, ignore_index=True)

In [26]:
index_levels.to_csv(r'index_levels.csv')